## this is the first file that should be ran. next would be relational notebook that uses pickle file

### 1. Setup imports and API key credentials

In [1]:
# imports and client instance creation with apikey
import os
import omdb
import pandas as pd
from omdb import OMDBClient
from datetime import datetime, timedelta
client = OMDBClient(apikey='e55a0e19')

# define input and output folder paths
input_folder = r'C:\My\Workspace\Python_Projects\Movies_Data\input_files'
output_folder = r'C:\My\Workspace\Python_Projects\Movies_Data\output_files'

user_created_data_folder = r'C:\My\Workspace\Python_Projects\Movies_Data\input_files\user_created_moviedata'
staging_folder = r'C:\My\Workspace\Python_Projects\Movies_Data\output_files\4_staging'
raw_folder = r'C:\My\Workspace\Python_Projects\Movies_Data\output_files\3_raw'
curated_folder = r'C:\My\Workspace\Python_Projects\Movies_Data\output_files\2_curated'

### 2. Define common functions

In [2]:
# helper function to search omdb for each film and return the unique imdb ID
# 7/22/23 may be able to add inventory_id here so that inventory file can be tied in with films file
def get_film_object(row):
    film_title = row['movie_title']
    film_year_released = str(row['release_year'])
    row_dict = row.to_dict()
    expanded_dict = {}
    try:
        film_object = client.search(film_title, year=film_year_released)
        # expanded_dict = row_dict.update(film_object[0])
        return film_object[0]
    except Exception as e:
        print(f"Skipping row: {row} - Error: {e}")
        return None

# helper function to extract the unique imdb ids from the list
def extract_column_to_list(df, column_name):
    return df[column_name].tolist()

### 3. MovieData Operations

In [3]:
# 7/16/23 adding a copy of this here even though it needs work -its a good starting point. This should be modified as a check for changes area so the API is only hit for new entries
# 7/17/23 in the long run this should turn into a staging table for get_film_object -> compare to existing -> fetch new

# first, check for current MovieData file so the API can avoid getting hit
file_name = 'data_list.pkl'
output_file = os.path.join(input_folder, file_name)

try:
    if os.path.exists(output_file):
        file_time = datetime.fromtimestamp(os.path.getmtime(output_file))
        current_time = datetime.now()
        time_difference = current_time - file_time
        if time_difference.days > 2:
            # File is more than 2 days old, perform necessary actions
            print("Current File is outdated. Go ahead and create a new one")
        else:
            # File is less than or equal to 2 days old, no action needed
            print("File is less than 2 days old. Consider using the existing one")
    else:
        # File does not exist, handle accordingly
        print("FILE DOES NOT EXIST! A new one will be created")
except Exception as e:
    # Exception occurred, handle the error
    print("An error occurred:", str(e))

FILE DOES NOT EXIST! A new one will be created


In [3]:
# create path and name variable
file_name = 'film_list.csv'
input_file = os.path.join(user_created_data_folder, file_name)
# read csv
csv_data = pd.read_csv(input_file, delimiter='|')

In [5]:
# create a dataframe and export of film_list as it since it contains other important data
# column_list = ["Inventory_Id", "Title","Year_Released","Watched", "4kUltra_Owned","Bluray_Owned","DVD_Owned","Stream_Owned"]
column_list = ["inventory_id", "movie_title","release_year","purchase_date", "watch_date", "4k_ultra_qty","bluray_qty","dvd_qty","stream_qty", "watch_flag"]
inventory_df = csv_data[column_list]
# create path and name variable
file_name = 'movie_inventory.csv'
output_file = os.path.join(staging_folder, file_name)

# this is to persist the dataframe
inventory_df.to_csv(output_file, sep='|', index=False)
print("stops")

stops
stops


In [6]:
# uses get_film_object helper function to find movie match
csv_data['film_object'] = csv_data.apply(get_film_object, axis=1)

In [8]:
# create path and name variable
file_name = 'movie_inventory_multiline.csv'
output_file = os.path.join(staging_folder, file_name)

# this is to persist the dataframe
csv_data.to_csv(output_file, sep='|', index=False)

In [7]:
# should insert the convert inflation lookup table here. Can mimic the code for the film_list.csv to movie_inventory.csv functionality
print("stop")

stop


In [18]:
# 7/16/23 adding this code from movies_data_nb_dev1a, but it could still use editing to keep only the essential parts

# would like to make a non-key version of movie-list.csv so it can keep a working movie_list.csv->Movie_Data_DW.movies_staging->power bi report for the time being
# continuation of above...this creates a list of dictionaries with 5 entries each (key, title, year, imdb_id, type, poster)
# film_data = [value for _, value in csv_data['film_object'].items()]
# value in csv_data['inventory_id'].items()
# try to append inventory_id since it is a list
film_data = [value for _, value in csv_data['film_object'].items()]
# creates a dataframe with five columns (title, year, imdb_id, type, poster)
full_df = pd.DataFrame(film_data)

column_list = ["title", "year", "imdb_id", "type", "poster"]

full_df = full_df[column_list]

# create path and name variable
file_name = 'movie_list.csv'
output_file = os.path.join(staging_folder, file_name)

# this is to persist the dataframe, may not be necessary in databricks
full_df.to_csv(output_file, sep='|', index=False)

KeyboardInterrupt: 

In [19]:
# uses extract_column_to_list helper function to isolate the imdb_id column
imdb_id_list = extract_column_to_list(full_df, 'imdb_id')

In [20]:
# create list of API endpoints for further lookups, example URL: https://www.omdbapi.com/?i=tt2382320&apikey=e55a0e19
url_list = []
import urllib.parse
for imdbId in imdb_id_list:
    url = 'https://www.omdbapi.com/?'
    params = {'i': imdbId, 'apikey': 'e55a0e19'}
    temp_url = str(url + urllib.parse.urlencode(params))
    url_list.append(temp_url)

In [21]:
# create csv of endpoints...can do single list or can apply to an existing dataframe
s_obj = pd.Series(url_list)
# create path and name variable
file_name = 'endpoint_list.csv'
output_file = os.path.join(staging_folder, file_name)
# Save the DataFrame to a CSV file with no leading 0 and no quoting of fields
s_obj.to_csv(output_file, header=False, index=False)

In [22]:
# could make a more raw form of the results here
import requests
# import json
data_list_raw = []
for api_url in url_list:
    resp = requests.get(api_url)
    resp.raise_for_status()
    data = resp.json()
    # for d in data["Ratings"]:
    #     temp_dict = {d["Source"].replace(" ", ""):d["Value"]}
    #     data.update(temp_dict.items()) # data is a dict object with 28 entries each pass through the loop
    data_list_raw.append(data) #data_list is a list of dictionaries that are created each pass

#create a dataframe from the list
data_list_raw_df = pd.DataFrame(data_list_raw)

#save the entire set to file
file_name = 'movie_list_raw_all_columns.json'
output_file = os.path.join(staging_folder, file_name)
data_list_raw_df.to_json(output_file, orient="table")

In [27]:
#save the entire set to file
file_name = 'movie_list_raw_all_columns.csv'
output_file = os.path.join(staging_folder, file_name)
data_list_raw_df.to_csv(output_file,sep='|', index=False)

In [23]:
# create a list where each row is all the movie data found from each key
import requests
# import json
data_list = []
for api_url in url_list:
    resp = requests.get(api_url)
    resp.raise_for_status()
    data = resp.json()
    for d in data["Ratings"]:
        temp_dict = {d["Source"].replace(" ", ""):d["Value"]}
        data.update(temp_dict.items()) # data is a dict object with 28 entries each pass through the loop
    data_list.append(data) #data_list is a list of dictionaries that are created each pass

#create a dataframe from the pre-loaded list
data_list_df = pd.DataFrame(data_list)

### 4. Export dataframe as is

In [24]:
import pickle

# save dictionary to data_list.pkl file
file_name = 'data_list.pkl'
output_file = os.path.join(staging_folder, file_name)
with open(output_file, 'wb') as fp:
    pickle.dump(data_list, fp)

In [25]:
#save the entire set to file
file_name = 'movie_list_all_columns.json'
output_file = os.path.join(staging_folder, file_name)
data_list_df.to_json(output_file, orient="table")

In [26]:
#save the entire set to file
file_name = 'movie_list_all_columns.csv'
output_file = os.path.join(staging_folder, file_name)
data_list_df.to_csv(output_file,sep='|', index=False)

### 5. Clean and conform the DataFrame

In [28]:
column_list=["Title","Year","Rated","Released","Runtime","Genre","Director","Writer","Actors","Plot","Language","Country","Awards","Poster", "Ratings", "Metascore","imdbRating","imdbVotes","InternetMovieDatabase","RottenTomatoes","Metacritic","imdbID","Type","DVD","BoxOffice","Production","Website"]
# create a dataframe object based on the previous one where only the relevant columns are used
data_list_df_partial = pd.DataFrame(data_list_df, columns=column_list)

In [ ]:
# convert these to snake case at some point
# column_list=["Title","Year","Rated","Released","Runtime","Genre","Director","Writer","Actors","Plot","Language","Country","Awards","Poster", "Ratings", "Metascore","imdbRating","imdbVotes","InternetMovieDatabase","RottenTomatoes","Metacritic","imdbID","Type","DVD","BoxOffice","Production","Website"]
# # create a dataframe object based on the previous one where only the relevant columns are used
# data_list_df_partial = pd.DataFrame(data_list_df, columns=column_list)

In [29]:
file_name = 'movie_list_detail.csv'
output_file = os.path.join(staging_folder, file_name)
data_list_df_partial.to_csv(output_file,sep='|', index=False)